# Kryptologie LAB - Übung 10 - DSA
- Signaturalgorithmus
- Basiert auf Diskreten Logarithmen
- Definiert im ’Digital Signature Standard’ (DSS)
- Benötigt globale Parameter p, q, g und Hashfunktion hash
- q ist Primzahl der Länge N
- p ist Primzahl der Länge L und darstellbar als p = kq + 1
- g ist ein Element aus Z∗p mit Ordnung q
- Erlaubte Längen (L,N) sind:
- (1024, 160),(2048, 224),(2048, 256),(3072, 256)
- Zusätzlich muss gelten N ≤ |hash|
- Hier: (L,N) = (1024, 160), hash = sha-1

In [1]:
import math
import hashlib

## Parametergenerierung

In [2]:
# from Übung 07
import random

# x^m mod n
def potModN(x, m, n):
    y = 1
    b = m
    while b != 0:
        if (b & 0x1) == 1:
            y = (y * x) % n
        x = (x*x) % n
        b = b >> 1
    return y

class RSA:
    small_primes = [1,7,11,13,17, 19,23,29]

    def eea(self, a,b):
        k = 0
        r = [a, b]
        s = [1, 0]
        t = [0, 1]
        while r[k+1] != 0:
            k += 1
            q_k = r[k-1] // r[k]
            r.append(r[k-1]-q_k*r[k])
            s.append(s[k-1]-q_k*s[k])
            t.append(t[k-1]-q_k*t[k])
        # ggt
        return (r[k], s[k], t[k])

    def millerRabin(self, n):
        # determine k and m
        temp = n-1
        k = 0
        while (temp % 2) == 0:
            temp //= 2
            k += 1
        m = temp
        # determine a
        a = random.randrange(2,n)
        b = potModN(a, m, n)
        if b == 1:
            return True
        for i in range(1, k+1):
            if b == n-1:
                return True
            else:
                b = potModN(b, 2, n)
        return False
        

    def testPrim(self, n):
        success = False
        for i in range(8):
            success = self.millerRabin(n)
            if success: return True
        return False

    def genPrim(self, min_z = 1000, max_z = 10**130):
        z = random.randint(min_z, max_z)
        hit = False
        i = 1
        while not hit:
            number = 30 * z + self.small_primes[i]
            hit = self.testPrim(number)
            i += 1
            if i == 8:
                i = 0
                z += 1
        return number
    
    def genKeyPair(self):
        # generate primes
        p = self.genPrim()
        q = self.genPrim()
        while min([p,q]) * 1.2 > max([p,q]):
            q = self.genPrim()
        # gen e
        n = p * q
        phi_n = (p-1) * (q-1)
        hit = False
        while not hit:
            e = random.randint(1000, 10**180)
            ggt, d, _ = self.eea(e, phi_n)
            hit = ggt == 1
        if d < 0:
            d = phi_n + d
        return ((e,n), (d, n))


    def encrypt(self, x, public):
        e, n = public
        return self.potModN(x,e,n)

    def decrypt(self, x, private):
        d, n = private
        return self.potModN(x,d,n)

## Parametergenerierung

In [3]:
# generate n ones in int
def ones(n):
    return (2**n)-1

# generate prime wit n bits
def genQ(n):
    rsa = RSA()
    return rsa.genPrim(min_z=math.floor((1<<(n-1)) / 30), max_z=math.ceil(ones(n) / 30))

def genParameters(l: int, n: int):
    rsa = RSA()
    q = genQ(n)
    hit = False
    tries = 0
    while not hit:
        # random h with 1 < h < p-1
        k = random.randint(2 ** (l - n), 2 ** (l - n + 1))
        p = q * k + 1
        # check if p is prime and and the correct length
        hit = rsa.testPrim(p) and p.bit_length() == l
        tries += 1
        if tries > 10000:
            print("Generating new q")
            q = genQ(n)
            tries = 0
    assert (p - 1) % q == 0
    assert q.bit_length() == n
    assert p.bit_length() == l
    print("q length: ", q.bit_length())
    print("p length: ", p.bit_length())

    # Bestimme g aus Z*_P mit Ordnung q
    g = 1
    while g == 1:
        h = random.randint(2, p-1)
        # g = h ^ ((p-1) / q) mod p
        g = potModN(h,(p-1) // q, p)
    print("g: ", g)
    return p, q, g


## Schlüsselgenerierung

In [4]:
def genKey(p, q, g):
    x = random.randint(2,q)
    y = potModN(g,x,p)
    return x,y

## Signieren

In [5]:
def sha1(input):
    return hashlib.sha1(input).digest()

# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [6]:
# Sign Algorithm with Inputs:
# m - message
# p - one of the primes
# q - other of the primes
# g - generator
# x - private key
def sign(m,p,q,g,x):
    hashedMessage = int_from_bytes(sha1(m))
    r = 0
    s = 0
    rsa = RSA()
    while r == 0 or s == 0:
        j = random.randint(2, q)
        r = potModN(g,j,p) % q
        one, invertedJ, _ = rsa.eea(j,q)
        assert one == 1
        s = (invertedJ * (hashedMessage + r * x)) % q
    return r,s

# Verify Algorithm
# m - message
# p - one of the primes
# q - other of the primes
# g - generator
# y - public key
# r - signature part 1
# s - signature part 2
def verify(m,p,q,g,y,r,s):
    hashedMessage = int_from_bytes(sha1(m))
    rsa = RSA()
    # Check intervals of r and s
    if 0 < r < q and 0 < s < q:
        _, w, _ = rsa.eea(s,q)
        u_1 = (hashedMessage * w) % q
        u_2 = (r*w) % q
        # v = (g^u1 * y^u2 mod p) mod q
        v = ((potModN(g,u_1,p) * potModN(y,u_2,p)) % p) % q
        if v == r:
            print("Valid signature")
            return True
        else:
            print("Invalid signature (v: ", v, " - r: ", r, ")")
            return False
    else:
        print("r and s in wrong intervalls")
        return False


In [7]:
p, q, g = genParameters(1024,160)
x,y = genKey(p,q,g)

q length:  160
p length:  1024
g:  34812612395891250039585782671029308420759872903547711834001001395583278187989384253500322833547947155537225572377417237744854481791962087277850249470792018763554509445598998637586431184925452137961147520846973033052710147325274599474220870650132058364882948826949600688317481215421661576239249740703355081856


In [8]:
message = b"Hallo, das ist ein Test"
r,s = sign(message,p,q,g,x)
print("r: ", r.bit_length())
print("s: ", s.bit_length())
verify(message,p,q,g,y,r,s)

r:  160
s:  158
Valid signature


True